## Composite Agents with Docker Container Computer

This notebook walks you through running a composed GUI agent using a Docker-based Computer and OpenRouter for the grounding model, paired with a planning model.

We'll use the model string:

- `"openrouter/z-ai/glm-4.5v+openai/gpt-5-nano"` (grounding + planning)

Grounding (left) generates actionable UI coordinates; planning (right) reasons and drives steps.

### Prerequisites

- Docker Desktop or Engine installed and running
- An OpenRouter account and API key (https://openrouter.ai/)
- (Optional) An OpenAI API key if using `openai/gpt-5-nano` for planning
- Python 3.12 environment with `cua-agent` installed

In [ ]:
# Install CUA Agent (and extras as needed)
!pip install -q "cua-agent[all]"

### Prepare a Docker Computer

We'll follow the documented Docker provider flow (see `docs/content/docs/computer-sdk/computers.mdx`).

If you don't have the image yet, either pull or build it locally. Run these in a terminal, not inside the notebook:

```bash
# Option 1: Pull from Docker Hub
docker pull trycua/cua-ubuntu:latest

# Option 2: Build locally (from repo root)
cd libs/kasm
docker build -t cua-ubuntu:latest .
```

### Set environment keys

- Get an OpenRouter API key at https://openrouter.ai/
- If using OpenAI for planning, set your OpenAI key as well
- You can input them here to set for this notebook session

In [ ]:
import os

OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY') or input('Enter your OPENROUTER_API_KEY: ').strip()
os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY

# Optional: if planning model uses OpenAI provider
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or input('(Optional) Enter your OPENAI_API_KEY (press Enter to skip): ').strip()
if OPENAI_API_KEY:
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Create a Docker Computer and a composed agent

This uses the documented Docker provider parameters: `os_type="linux"`, `provider_type="docker"`, plus `image` and `name`.

In [ ]:
import asyncio
from computer import Computer
from agent import ComputerAgent

async def main():
    # Launch & connect to a Docker container running the Computer Server
    async with Computer(
        os_type='linux',
        provider_type='docker',
        image='trycua/cua-ubuntu:latest',
        name='my-cua-container'
    ) as computer:
        agent = ComputerAgent(
            model='openrouter/z-ai/glm-4.5v+openai/gpt-5-nano',
            tools=[computer],
            trajectory_dir='trajectories' # Save agent trajectory (screenshots, api calls)
        )

        # Simple task to verify end-to-end
        async for _ in agent.run('Open a browser and go to example.com'):
            pass

asyncio.run(main())

### Notes

- Grounding (OpenRouter `z-ai/glm-4.5v`) + Planning (OpenAI `gpt-5-nano`) can be swapped for other providers/models.
- If you prefer to avoid OpenAI, choose a planning model on OpenRouter and update the model string accordingly.
- Be sure the planning model supports `vision` input and the `tools` parameter.
- The agent emits normalized Agent Responses across providers.